In [2]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence

# init embedding
glove_embedding = WordEmbeddings('de-crawl')

2021-11-27 14:58:26,234 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-crawl-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpvrfy_91w


100%|██████████| 1199998928/1199998928 [25:11<00:00, 794120.84B/s] 

2021-11-27 15:23:37,957 copying /tmp/tmpvrfy_91w to cache at /home/mendu/.flair/embeddings/de-crawl-fasttext-300d-1M.vectors.npy


2021-11-27 15:23:48,276 removing temp file /tmp/tmpvrfy_91w
2021-11-27 15:23:48,906 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-crawl-fasttext-300d-1M not found in cache, downloading to /tmp/tmpc1nnxa9r


100%|██████████| 42677439/42677439 [00:51<00:00, 822156.29B/s] 

2021-11-27 15:24:41,419 copying /tmp/tmpc1nnxa9r to cache at /home/mendu/.flair/embeddings/de-crawl-fasttext-300d-1M


2021-11-27 15:24:41,508 removing temp file /tmp/tmpc1nnxa9r


In [26]:
def embed(word):
    # create sentence.
    sentence = Sentence(word)

    # embed a sentence using glove.
    glove_embedding.embed(sentence)

    # now check out the embedded tokens.
    for token in sentence:
        #print(token)
        #print(token.embedding)
        torch_tensor = token.embedding
        np_arr = torch_tensor.cpu().detach().numpy()
        return(np_arr)    

In [27]:
embed('Alles')

array([-0.0312,  0.0246,  0.0563, -0.    ,  0.0175, -0.06  , -0.0356,
       -0.0589,  0.0037,  0.0257, -0.0082, -0.048 ,  0.0091, -0.0106,
       -0.0848, -0.1172,  0.0391, -0.0382, -0.0252, -0.0812,  0.0134,
        0.0119,  0.027 ,  0.015 ,  0.0026,  0.0277,  0.0092,  0.0222,
        0.0159,  0.002 ,  0.0974, -0.0257, -0.0019, -0.0929,  0.0111,
       -0.0111, -0.0229,  0.1161, -0.0345, -0.1042,  0.0099, -0.054 ,
        0.0373, -0.0115, -0.0523, -0.0097,  0.0188,  0.0204,  0.0048,
       -0.0762, -0.0294, -0.0072, -0.0143, -0.0305, -0.0112,  0.0083,
       -0.0064,  0.0513, -0.0103, -0.102 , -0.0358, -0.0458, -0.0349,
        0.0992,  0.0205, -0.0247,  0.0157, -0.0332,  0.009 , -0.1691,
       -0.0371, -0.0963, -0.0576,  0.0081, -0.0069,  0.0516,  0.041 ,
        0.0151,  0.0144, -0.0216,  0.0075,  0.0487, -0.0406, -0.0249,
       -0.05  ,  0.1426, -0.0039, -0.0336,  0.0233, -0.0105,  0.0521,
       -0.0111, -0.0382,  0.0133,  0.0094,  0.0949, -0.0411, -0.0459,
        0.0366, -0.0

In [30]:
#gloveFile = "glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

import re
from nltk.corpus import stopwords
import pandas as pd


def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("german"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(embed(word1), embed(word2)))

def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([embed(word) for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([embed(word) for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')

def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5,5)) 
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    print(cosine_distance_wordembedding_method(s1, s2))
    return ax_blue

ss1 = 'AI Artificial Intelligence Machine Learning künstliche Intelligenz'
ss2 = 'Der #ModernWorkplace als #DigitalColleague Ihrer Mitarbeiter - Unsinn oder doch bald Realität? Im #Webinar am 08.05. beantwortet unser Kollege @Twalz Ihre Fragen rund um den Einsatz von #ChatBots und #KI. Jetzt kostenfrei anmelden! https://t.co/bzvXKbHP9e /tir'

#model = loadGloveModel(gloveFile)
heat_map_matrix_between_two_sentences(ss1,ss2)

Word Embedding method with a cosine distance asses that our two sentences are similar to 72.58 %
None
